In [ ]:
""" Demonstrates the easy of integration of a custom layer """
import math
import torch
import torch.nn as nn
import numpy as np

In [ ]:
class MyLinearLayer(nn.Module):
    """ Custom Linear layer but mimics a standard linear layer """
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        # weights = torch.Tensor(size_out, size_in)

        # initialize weights and biases
        weights = torch.randn(
            self.size_in, self.size_out, dtype=torch.cdouble
        ) / math.sqrt(self.size_in)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        
        bias = torch.unsqueeze(
            torch.zeros(size_out, dtype=torch.cdouble, requires_grad=True), 0
        )
        self.bias = nn.Parameter(bias)

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights)
        return torch.add(w_times_x, self.bias)  # w times x + b
        #return x @ self.weights + self.bias

In [ ]:
class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = MyLinearLayer(2, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
torch.manual_seed(0)  #  for repeatable results
model = BasicModel()

In [ ]:
model.linear.weights

Parameter containing:
tensor([[ 0.7705-0.1467j],
        [-1.0894+0.2842j]], dtype=torch.complex128, requires_grad=True)

## XOR Example

In [ ]:
x = torch.Tensor([[0., 0.],
               [0., 1.],
               [1., 0.],
               [1., 1.]])

y = torch.Tensor([0., 1., 1., 0.]).reshape(x.shape[0], 1)

In [ ]:
x = x.type(torch.cdouble)
print('Forward computation thru model:', model(x))

Forward computation thru model: tensor([[ 0.0000+0.0000j],
        [-1.0894+0.2842j],
        [ 0.7705-0.1467j],
        [-0.3189+0.1375j]], dtype=torch.complex128, grad_fn=<AddBackward0>)


In [ ]:
print('Forward computation thru model:', model(x))

Forward computation thru model: tensor([[ 0.0000+0.0000j],
        [-1.0894+0.2842j],
        [ 0.7705-0.1467j],
        [-0.3189+0.1375j]], dtype=torch.complex128, grad_fn=<AddBackward0>)


In [ ]:
#PyTorch
class Complex_RMSE(nn.Module):
    def __init__(self, categories: int = 2, periodicity: int = 2):
        super(Complex_RMSE, self).__init__()
        self.categories = categories
        self.periodicity = periodicity

    def forward(self, inputs, targets): 
        target_angles = self.class2angle(targets)
        predicted_angles = torch.remainder(inputs.angle() + 2 * np.pi, 2 * np.pi)
        # z = torch.abs(predicted) * torch.exp(1.j * predicted_angles)
        # error calculation
        errors = torch.exp(1.0j * target_angles) - torch.exp(1.0j * predicted_angles)
        if self.periodicity > 1:
            # select smallest error
            idx = torch.argmin(torch.abs(errors), dim=1, keepdim=True)
            return errors.gather(1, idx).mean()
        else:
            return errors.mean()

    def class2angle(self, actual: torch.tensor) -> torch.tensor:
        # Returns a new tensor with an extra dimension
        if actual.size().__len__() == 1:
            actual = torch.unsqueeze(actual, 1)

        # Class to several angles due to periodicity using bisector
        return (
            (self.categories * torch.arange(self.periodicity) + actual + 0.5)
            / (self.categories * self.periodicity)
            * 2
            * np.pi
        )

In [ ]:
from torch.autograd import Function

#######################################################
class MyComplexRMSELoss(Function):

    @staticmethod
    def forward(ctx, y_pred, y):
        target_angles = class2angle(y)
        predicted_angles = torch.remainder(inputs.angle() + 2 * np.pi, 2 * np.pi)
        ctx.save_for_backward(y_pred, y)
        return ( (y - y_pred)**2 ).mean()
    
    @staticmethod
    def backward(ctx, grad_output):
        y_pred, y = ctx.saved_tensors
        grad_input = 2 * (y_pred - y) / y_pred.shape[0]        
        return grad_input, None

    def class2angle(self, actual: torch.tensor) -> torch.tensor:
        # Returns a new tensor with an extra dimension
        if actual.size().__len__() == 1:
            actual = torch.unsqueeze(actual, 1)

        # Class to several angles due to periodicity using bisector
        return (
            (self.categories * torch.arange(self.periodicity) + actual + 0.5)
            / (self.categories * self.periodicity)
            * 2
            * np.pi
        )
    
#######################################################

In [ ]:
# criterion = torch.nn.MSELoss(reduction='sum')
# criterion = MSEC(categories=2, periodicity=2)
criterion = Complex_RMSE(categories=2, periodicity=2)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [ ]:
y_pred = model(x)

In [ ]:
y_pred

tensor([[ 0.0000+0.0000j],
        [-1.0894+0.2842j],
        [ 0.7705-0.1467j],
        [-0.3189+0.1375j]], dtype=torch.complex128, grad_fn=<AddBackward0>)

In [ ]:
loss = criterion(y_pred, y)
loss

tensor(-0.0241-0.1153j, dtype=torch.complex128, grad_fn=<MeanBackward0>)

In [ ]:
loss.backward()

In [ ]:
optimizer.step()

In [ ]:

for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        # print(t, torch.abs(loss))
        print(t, torch.square(
                np.pi - torch.abs(torch.abs(loss) - np.pi)
            ))
        

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 tensor(0.2840, dtype=torch.float64, grad_fn=<PowBackward0>)
199 tensor(0.2838, dtype=torch.float64, grad_fn=<PowBackward0>)
299 tensor(0.2836, dtype=torch.float64, grad_fn=<PowBackward0>)
399 tensor(0.2834, dtype=torch.float64, grad_fn=<PowBackward0>)
499 tensor(0.2833, dtype=torch.float64, grad_fn=<PowBackward0>)
599 tensor(0.2831, dtype=torch.float64, grad_fn=<PowBackward0>)
699 tensor(0.2829, dtype=torch.float64, grad_fn=<PowBackward0>)
799 tensor(0.2827, dtype=torch.float64, grad_fn=<PowBackward0>)
899 tensor(0.2825, dtype=torch.float64, grad_fn=<PowBackward0>)
999 tensor(0.2824, dtype=torch.float64, grad_fn=<PowBackward0>)
1099 tensor(0.2822, dtype=torch.float64, grad_fn=<PowBackward0>)
1199 tensor(0.2820, dtype=torch.float64, grad_fn=<PowBackward0>)
1299 tensor(0.2818, dtype=torch.float64, grad_fn=<PowBackward0>)
1399 tensor(0.2817, dtype=torch.float64, grad_fn=<PowBackward0>)
1499 tensor(0.2815, dtype=torch.float64, grad_fn=<PowBackward0>)
1599 tensor(0.2813, dtype=torch.floa

AttributeError: 'BasicModel' object has no attribute 'string'

In [ ]:
""" Demonstrates the easy of integration of a custom layer """
import math
import torch
import torch.nn as nn
import numpy as np

class MyLinearLayer(nn.Module):
    """ Custom Linear layer but mimics a standard linear layer """
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)  # bias init

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b


class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 128, 3)
        # self.linear = nn.Linear(256, 2)
        self.linear = MyLinearLayer(256, 2)

    def forward(self, x):
        x = self. conv(x)
        x = x.view(-1, 256)
        return self.linear(x)

torch.manual_seed(0)  #  for repeatable results
basic_model = BasicModel()
inp = np.array([[[[1,2,3,4],  # batch(=1) x channels(=1) x height x width
                  [1,2,3,4],
                  [1,2,3,4]]]])
x = torch.tensor(inp, dtype=torch.float)
print('Forward computation thru model:', basic_model(x))

Forward computation thru model: tensor([[ 1.2722, -0.6224]], grad_fn=<AddBackward0>)


In [ ]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(basic_model.parameters(), lr=1e-6)

In [ ]:
y_pred = basic_model(x)
# Compute and print loss
loss = criterion(y_pred, y)

In [ ]:
y_pred

tensor([[ 1.2722, -0.6224]], grad_fn=<AddBackward0>)

In [ ]:
loss.backward()

RuntimeError: grad can be implicitly created only for scalar outputs